In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
train_data= pd.read_csv("titanic/train.csv",sep=",")
test_data=pd.read_csv("titanic/test.csv",sep=",")

In [3]:
%pip install kaggle


Note: you may need to restart the kernel to use updated packages.


In [4]:
train_data= pd.read_csv("titanic/train.csv",sep=",")
test_data=pd.read_csv("titanic/test.csv",sep=",")

In [5]:
passenger_id=test_data['PassengerId']
print(passenger_id)

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64


In [6]:
full_data=pd.concat([train_data,test_data],ignore_index=True)
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [7]:
#i am droping the name and passengerid and also ticket and cabin those factors wont help in deciding of survivability
full_data.drop(['Name','Ticket','PassengerId','Cabin'],axis=1,inplace=True)


In [8]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    float64
 1   Pclass    1309 non-null   int64  
 2   Sex       1309 non-null   object 
 3   Age       1046 non-null   float64
 4   SibSp     1309 non-null   int64  
 5   Parch     1309 non-null   int64  
 6   Fare      1308 non-null   float64
 7   Embarked  1307 non-null   object 
dtypes: float64(3), int64(3), object(2)
memory usage: 81.9+ KB


In [9]:
#after the dropping we are creting a new row called familiy
full_data['family_size']=train_data['Parch']+train_data['SibSp']
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Survived     891 non-null    float64
 1   Pclass       1309 non-null   int64  
 2   Sex          1309 non-null   object 
 3   Age          1046 non-null   float64
 4   SibSp        1309 non-null   int64  
 5   Parch        1309 non-null   int64  
 6   Fare         1308 non-null   float64
 7   Embarked     1307 non-null   object 
 8   family_size  891 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 92.2+ KB


In [10]:
full_data['Age'].fillna(full_data['Age'].median(),inplace=True)
full_data['Fare'].fillna(full_data['Fare'].median(),inplace=True)
full_data['Embarked'].fillna(full_data['Embarked'].mode()[0],inplace=True)

C:\Users\harsh\AppData\Local\Temp\ipykernel_3852\1812247621.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_data['Age'].fillna(full_data['Age'].median(),inplace=True)
C:\Users\harsh\AppData\Local\Temp\ipykernel_3852\1812247621.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
full_data['encoded_sex']=le.fit_transform(full_data['Sex'])
full_data['encoded_embark']=le.fit_transform(full_data['Embarked'])

In [12]:
full_data.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,encoded_sex,encoded_embark
0,0.0,3,male,22.0,1,0,7.2500,S,1.0,1,2
1,1.0,1,female,38.0,1,0,71.2833,C,1.0,0,0
2,1.0,3,female,26.0,0,0,7.9250,S,0.0,0,2
3,1.0,1,female,35.0,1,0,53.1000,S,1.0,0,2
4,0.0,3,male,35.0,0,0,8.0500,S,0.0,1,2
5,0.0,3,male,28.0,0,0,8.4583,Q,0.0,1,1
6,0.0,1,male,54.0,0,0,51.8625,S,0.0,1,2
7,0.0,3,male,2.0,3,1,21.0750,S,4.0,1,2
8,1.0,3,female,27.0,0,2,11.1333,S,2.0,0,2
9,1.0,2,female,14.0,1,0,30.0708,C,1.0,0,0


In [13]:
full_data.drop(['SibSp','Parch','Sex','Embarked'],inplace=True,axis=1)

In [14]:
full_data.head(10)

,Survived,Pclass,Age,Fare,family_size,encoded_sex,encoded_embark
0,0.0,3,22.0,7.2500,1.0,1,2
1,1.0,1,38.0,71.2833,1.0,0,0
2,1.0,3,26.0,7.9250,0.0,0,2
3,1.0,1,35.0,53.1000,1.0,0,2
4,0.0,3,35.0,8.0500,0.0,1,2
5,0.0,3,28.0,8.4583,0.0,1,1
6,0.0,1,54.0,51.8625,0.0,1,2
7,0.0,3,2.0,21.0750,4.0,1,2
8,1.0,3,27.0,11.1333,2.0,0,2
9,1.0,2,14.0,30.0708,1.0,0,0


In [20]:
train = full_data[:len(train_data)]
test = full_data[len(train_data):]


In [22]:
x=train.drop(['Survived'],axis=1)
y=train['Survived']
x_test=test.drop('Survived',axis=1)
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=42)

In [25]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}
grid_search=GridSearchCV(RandomForestClassifier(random_state=42),param_grid,cv=5,scoring='accuracy',n_jobs=1,verbose=1)
grid_search.fit(x_train,y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=1,
             param_grid={'max_depth': [5, 10, None],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='accuracy', verbose=1)

In [28]:
from sklearn.metrics import accuracy_score
print("best parameters:",grid_search.best_params_)
print("best cv score:",grid_search.best_score_)
best_model=grid_search.best_estimator_
valuation_preds=best_model.predict(x_val)
print("valuation accuracy:",accuracy_score(y_val,valuation_preds))

best parameters: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
best cv score: 0.8328080370333891
valuation accuracy: 0.8044692737430168


In [29]:
final_prediction=best_model.predict(x_test)

In [30]:
print(final_prediction)

[0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.

In [35]:
prediction_titanic=pd.DataFrame({
    'passengerID':passenger_id,
    'Survived':final_prediction,
    
})

In [36]:
prediction_titanic.to_csv('submission.csv',index=False)